Escuela Politécnica Nacional<br>


Facultad de Ingeniería en Sistemas<br>

Ingeniería en Ciencias de la Computación<br>
<br>
RECUPERACION DE INFORMACIÓN - ICCD753<br>

<br>
Tema: Modelo Booleano de RI<br>
Fecha de Entrega: 05-01-2022 <br>
<br>
Integrantes: <br>
Luis Ernesto Almeida Zambrano<br>
            Edison Daniel Cabrera Pabón

Ejemplo Uno

In [15]:
# implementation of vector space model for document retrieval

import pandas
# module to read the contents of the file from a csv file

from contextlib import redirect_stdout
# module to redirect the output to a text file

import math
# module to perform mathematical functions

terms = []
# list to store the terms present in the documents

keys = []
# list to store the names of the documents

vec_Dic = {}
# dictionary to store the name of the document and the weight as list

dicti = {}
# dictionary to store the name of the document and the terms present in it as a
# vector

dummy_List = []
# list for performing some operations and clearing them

term_Freq = {}
# dictionary to store the term and the number of times of its occurrence in the
# documents

idf = {}
# dictionary to store the term and the inverse document frequency

weight = {}
# dictionary to store the term and the weight which is the product of term
# frequency and inverse document frequency


def filter(documents, rows, cols):
	'''function to read and separate the name of the documents and the terms
	present in it to a separate list from the data frame and also create a
	dictionary which has the name of the document as key and the terms present
	in it as the list of strings which is the value of the key'''

	for i in range(rows):
		for j in range(cols):
			# traversal through the data frame

			if(j == 0):
				# first column has the name of the document in the csv file
				keys.append(documents.loc[i].iat[j])
			else:
				dummy_List.append(documents.loc[i].iat[j])
				# dummy list to update the terms in the dictionary

				if documents.loc[i].iat[j] not in terms:
					# add the terms to the list if it is not present else continue
					terms.append(documents.loc[i].iat[j])

		copy = dummy_List.copy()
		# copying the the dummy list to a different list

		dicti.update({documents.loc[i].iat[0]: copy})
		# adding the key value pair to a dictionary

		dummy_List.clear()
		# clearing the dummy list


def compute_Weight(doc_Count, cols):
	'''Function to compute the weight for each of the terms in the document.
	Here the weight is calculated with the help of term frequency and
	inverse document frequency'''

	for i in terms:
		# initially adding all the elements into the dictionary and initialising
		# the values as zero
		if i not in term_Freq:
			term_Freq.update({i: 0})

	for key, value in dicti.items():
		# to get the number of occurrence of each terms
		for k in value:
			if k in term_Freq:
				term_Freq[k] += 1
				# value incremented by one if the term is found in the documents

	idf = term_Freq.copy()
	# copying the term frequency dictionary to a dictionary named idf which is
	# further neede for computation

	for i in term_Freq:
		term_Freq[i] = term_Freq[i]/cols
		# term frequency is number of occurrence divided by total number of
		# documents

	for i in idf:
		if idf[i] != doc_Count:
			idf[i] = math.log2(cols / idf[i])
			# inverse document frequency log of total number of documents divided
			# by number of occurrence of the terms
		else:
			idf[i] = 0
			# this is to avoid the zero division error

	for i in idf:
		weight.update({i: idf[i]*term_Freq[i]})
		# weight is the product of term frequency and the inverse document
		# frequency

	for i in dicti:
		for j in dicti[i]:
			dummy_List.append(weight[j])

		copy = dummy_List.copy()
		vec_Dic.update({i: copy})
		dummy_List.clear()
		# above operations performed to get the dictionary of weighted vector
		# for each of the documents


def get_Weight_For_Query(query):
	'''function to get the weight for each terms present in the query, here we
	consider the term frequency as the weight of the terms'''

	query_Freq = {}
	# initialisation of the dictionary with query terms as key and its weight as
	# the values

	for i in terms:
		# initially adding all the elements into the dictionary and initialising
		# the values as zero
		if i not in query_Freq:
			query_Freq.update({i: 0})

	for val in query:
		# to get the number of occurrence of each terms
		if val in query_Freq:
			query_Freq[val] += 1
			# value incremented by one if the term is found in the documents

	for i in query_Freq:
		query_Freq[i] = query_Freq[i] / len(query)
		# term frequency obtained by dividing the number of occurrence of terms by
		# total number of terms in the query

	return query_Freq
	# return the dictionary in which the key is the term and the value is the
	# weight


def similarity_Computation(query_Weight):
	''' Function to calculate the similarity measure in which the weight of the
	query and the document is multiplied in the numerator and the the weight is
	squared and squareroot is taken the weights of the query and document'''

	numerator = 0
	denomi1 = 0
	denomi2 = 0
	# initialisation of the variables with zero which is needed for computation

	similarity = {}
	# initialisation of dictionary which has the name of document as key and the
	# similarity measure as value

	for document in dicti:
		for terms in dicti[document]:
			# cosine similarity is calculated

			numerator += weight[terms] * query_Weight[terms]
			denomi1 += weight[terms] * weight[terms]
			denomi2 += query_Weight[terms] * query_Weight[terms]
			# the summation values of the weight is calculated and later they are
			# divided

		if denomi1 != 0 and denomi2 != 0:
			# to avoid the zero division error

			simi = numerator / (math.sqrt(denomi1) * math.sqrt(denomi2))
			similarity.update({document: simi})
			#dictionary is updated

			numerator = 0
			denomi2 = 0
			denomi1 = 0
			# reinitialisation of the variables to zero

	return (similarity)
	# the dictionary containing similarity measure as the value


def prediction(similarity, doc_count):
	'''Function to predict the document which is relevant to the query '''

	with open('Salida.txt', 'w') as f:
		with redirect_stdout(f):
			# to redirect the output to a text file

			ans = max(similarity, key=similarity.get)
			print(ans, "is the most relevant document")
			# to print the name of the document which is most relevant

			print("ranking of the documents")
			for i in range(doc_count):
				ans = max(similarity, key=lambda x: similarity[x])
				print(ans, "rank is", i+1)
				# to print the document name and its rank

				similarity.pop(ans)


def main():
	documents = pandas.read_csv(r'DocumentoUno.csv')
	# to read the data from the csv file as a dataframe

	rows = len(documents)
	# to get the number of rows

	cols = len(documents.columns)
	# to get the number of columns

	filter(documents, rows, cols)
	# function call to read and separate the name of the documents and the terms
	# present in it to a separate list from the data frame and also create a
	# dictionary which has the name of the document as key and the terms present
	# in it as the list of strings which is the value of the key

	compute_Weight(rows, cols)
	# Function to compute the weight for each of the terms in the document.
	# Here the weight is calculated with the help of term frequency and inverse
	# document frequency

	print("Enter the query")
	query = input()
	# to get the query input from the user, the below input is given for obtaining
	# the output as in output.txt file
	# one three three

	query = query.split(' ')
	# spliting the query as a list of strings

	query_Weight = get_Weight_For_Query(query)
	# function call to get the weight for each terms present in the query, here we
	# consider the term frequency as the weight of the terms'''

	similarity = similarity_Computation(query_Weight)
	# Function call to calculate the similarity measure in which the weight of the
	# query and the document is multiplied in the numerator and the weight is
	# squared and squareroot is taken the weights of the query and document

	prediction(similarity, rows)
	# Function call to predict the document which is relevant to the query


main()


Enter the query
hockey


ValueError: ignored

Ejemplo Dos


Step-1 Importing the necessary libraries


In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys
Stopwords = set(stopwords.words('english'))

LookupError: ignored

Step-2 Finding the set of unique words from all documents of the data set

In [ ]:
all_words = []
dict_global = {}
file_folder = 'data/*'
idx = 1
files_with_index = {}
for file in glob.glob(file_folder):
    print(file)
    fname = file
    file = open(file , "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = os.path.basename(fname)
    idx = idx + 1
    
unique_words_all = set(dict_global.keys())

Step-3 Implementing helper functions

In [ ]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq
def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)
        
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,'',text)
    return text_returned

Step-4 Defining the linked list

In [ ]:
class Node:
    def __init__(self ,docId, freq = None):
        self.freq = freq
        self.doc = docId
        self.nextval = None
    
class SlinkedList:
    def __init__(self ,head = None):
        self.head = head

Step-5 Making a linkedlist for each word and storing all the nodes (containing the file name and frequency of the respective word ) in the linkedlist.

In [ ]:
linked_list_data = {}
for word in unique_words_all:
    linked_list_data[word] = SlinkedList()
    linked_list_data[word].head = Node(1,Node)
word_freq_in_doc = {}
idx = 1
for file in glob.glob(file_folder):
    file = open(file, "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    word_freq_in_doc = finding_all_unique_words_and_freq(words)
    for word in word_freq_in_doc.keys():
        linked_list = linked_list_data[word].head
        while linked_list.nextval is not None:
            linked_list = linked_list.nextval
        linked_list.nextval = Node(idx ,word_freq_in_doc[word])
    idx = idx + 1

Step -6 Query processing and output generation

In [ ]:
query = input('Enter your query:')
query = word_tokenize(query)
connecting_words = []
cnt = 1
different_words = []
for word in query:
    if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
        different_words.append(word.lower())
    else:
        connecting_words.append(word.lower())
print(connecting_words)
total_files = len(files_with_index)
zeroes_and_ones = []
zeroes_and_ones_of_all_words = []
for word in (different_words):
    if word.lower() in unique_words_all:
        zeroes_and_ones = [0] * total_files
        linkedlist = linked_list_data[word].head
        print(word)
        while linkedlist.nextval is not None:
            zeroes_and_ones[linkedlist.nextval.doc - 1] = 1
            linkedlist = linkedlist.nextval
        zeroes_and_ones_of_all_words.append(zeroes_and_ones)
    else:
        print(word," not found")
        sys.exit()
print(zeroes_and_ones_of_all_words)
for word in connecting_words:
    word_list1 = zeroes_and_ones_of_all_words[0]
    word_list2 = zeroes_and_ones_of_all_words[1]
    if word == "and":
        bitwise_op = [w1 & w2 for (w1,w2) in zip(word_list1,word_list2)]
        zeroes_and_ones_of_all_words.remove(word_list1)
        zeroes_and_ones_of_all_words.remove(word_list2)
        zeroes_and_ones_of_all_words.insert(0, bitwise_op);
    elif word == "or":
        bitwise_op = [w1 | w2 for (w1,w2) in zip(word_list1,word_list2)]
        zeroes_and_ones_of_all_words.remove(word_list1)
        zeroes_and_ones_of_all_words.remove(word_list2)
        zeroes_and_ones_of_all_words.insert(0, bitwise_op);
    elif word == "not":
        bitwise_op = [not w1 for w1 in word_list2]
        bitwise_op = [int(b == True) for b in bitwise_op]
        zeroes_and_ones_of_all_words.remove(word_list2)
        zeroes_and_ones_of_all_words.remove(word_list1)
        bitwise_op = [w1 & w2 for (w1,w2) in zip(word_list1,bitwise_op)]
zeroes_and_ones_of_all_words.insert(0, bitwise_op);
        
files = []    
print(zeroes_and_ones_of_all_words)
lis = zeroes_and_ones_of_all_words[0]
cnt = 1
for index in lis:
    if index == 1:
        files.append(files_with_index[cnt])
    cnt = cnt+1
    
print(files)